In [1]:
!wget https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json

--2024-06-23 02:17:58--  https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json [following]
--2024-06-23 02:17:58--  https://raw.githubusercontent.com/alexeygrigorev/llm-rag-workshop/main/notebooks/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.3’

documents.json.3    100%[===================>] 642.90K  --.-KB/s    in 0.01s   

2024-06-23 02:17:59 (60.1 MB/s) - ‘documents.json.3’ saved [65

In [2]:
!head 'documents.json'

[
  {
    "course": "data-engineering-zoomcamp",
    "documents": [
      {
        "text": "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  \u201cOffice Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon\u2019t forget to register in DataTalks.Club's Slack and join the channel.",
        "section": "General course-related questions",
        "question": "Course - When will the course start?"
      },
      {


In [15]:
import os
import json
from groq import Groq

with open('./documents.json', 'rt') as f_in:
    documents_file = json.load(f_in)

documents = []

for course in documents_file:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [16]:
len(documents)

948

In [3]:
documents[0].keys()

dict_keys(['text', 'section', 'question', 'course'])

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info()

ObjectApiResponse({'name': '9e7f4dadb1a6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'BhZLD2LURZykNtT6I7hu1w', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
response = es.indices.create(index=index_name, body=index_settings)

response

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

/home/codespace/.local/share/virtualenvs/llm-zoomcamp-workshop-H1KjvqNQ/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:21<00:00, 45.01it/s]


In [21]:
def retrieve_documents(query, index_name="course-questions", max_results=5):
    search_query = {
        "size": max_results,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es.search(index=index_name, body=search_query)
    documents = [hit['_source'] for hit in response['hits']['hits']]
    return documents

In [22]:
user_question = "How do I join the course after it has started?"

response = retrieve_documents(user_question)

for doc in response:
    print(f"Section: {doc['section']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:60]}...\n")

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to su...

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishe...

Section: General course-related questions
Question: Course - What can I do before the course starts?
Answer: You can start by installing and setting up all the dependenc...

Section: General course-related questions
Question: How do I use Git / GitHub for this course?
Answer: After you create a GitHub account, you should clone the cour...

Section: Workshop 1 - dlthub
Question: How do I install the necessary dependencies to run the code?
Answer: Answer: To run the provided code, ensure that the 'dlt[duckd...



In [30]:
context_template = """
Section: {section}
Question: {question}
Answer: {text}
""".strip()

prompt_template = """
You're a course teaching assistant.
Answer the user QUESTION based on CONTEXT - the documents retrieved from our FAQ database.
Don't use other information outside of the provided CONTEXT.  

QUESTION: {user_question}

CONTEXT:

{context}
""".strip()


def build_context(documents):
    context_result = ""
    
    for doc in documents:
        doc_str = context_template.format(**doc)
        context_result += ("\n\n" + doc_str)
    
    return context_result.strip()


def build_prompt(user_question, documents):
    context = build_context(documents)
    prompt = prompt_template.format(
        user_question=user_question,
        context=context
    )
    return prompt

def ask_groq(prompt, model="llama3-8b-8192"):
    client = Groq(
        api_key=os.environ.get("GROQ_API_KEY"),
    )
    
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
    )

    print(chat_completion.choices[0].message.content)

def qa_bot(user_question):
    context_docs = retrieve_documents(user_question)
    prompt = build_prompt(user_question, context_docs)
    answer = ask_groq(prompt)

In [31]:
qa_bot("I'm getting invalid reference format: repository name must be lowercase")

Your error is related to Docker, and more specifically, to the mounting of volumes. The error message is "invalid reference format: repository name must be lowercase". 

From the provided context, it seems that the issue might be caused by the fact that the repository name is not in lowercase. However, there is no specific information about how to correct this issue in the provided text. It seems that the provided answers only describe how to mount volumes in Docker, but do not mention specifically how to handle the error "invalid reference format: repository name must be lowercase".

If you are still experiencing this issue, you might want to check if the repository name is indeed the cause of the error and try to lowercase the name.


In [32]:
qa_bot("I can't connect to postgres port 5432, my password doesn't work")

Based on the CONTEXT, my answer to your question is:

I can't connect to postgres port 5432, my password doesn't work.

The error "FATAL: password authentication failed for user 'root'" means that the password for the 'root' user is incorrect. 

Please try the following solutions:

- Check if you have a local Postgres installation on your computer. If yes, use a different port, like 5431, when creating the docker container, as in: -p 5431:5432
- Then, use this port when connecting to pgcli, as shown below:
  pgcli -h localhost -p 5431 -u root -d ny_taxi

If you still encounter the error, make sure there is no service in Windows running Postgres and stop that service.


In [33]:
qa_bot("how can I run kafka?")

Based on the provided CONTEXT, to run Kafka, please refer to the answer: Section: Module 6: streaming with kafka

Question: Java Kafka: How to run producer/consumer/kstreams/etc in terminal
Answer: In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java


In [34]:
qa_bot("can i still able to join the course after it starts?")

Based on the CONTEXT provided, I can answer your QUESTION:

Yes, you can still join the course after it starts. According to the answer from Section: General course-related questions, Question: Course - Can I still join the course after the start date?, "Yes, even if you don't register, you're still eligible to submit the homeworks."
